In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git


In [ ]:
!pip install -q datasets scipy wandb
!pip install -q trl
!pip install -q ipywidgets==7.7.1

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!nvidia-smi

In [ ]:
import torch
print(torch.version.cuda)

In [ ]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

In [ ]:
fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset_name = 'Amod/mental_health_counseling_conversations'
dataset = load_dataset(dataset_name, split="train")

In [ ]:
print(dataset)

In [ ]:
print(dataset[0])

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig



In [ ]:
base_model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)



In [ ]:
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config)

In [ ]:
!pip install sentencepiece

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=1024,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
inputs = tokenizer("Qui est l'actuel président de la République française ?", return_tensors="pt")
output_generate=model.generate(**inputs,max_new_tokens=20, return_dict_in_generate=True, output_scores=True)
transition_scores = model.compute_transition_scores(output_generate.sequences, output_generate.scores, normalize_logits=True)



In [ ]:
import numpy as np

input_length = 1 if model.config.is_encoder_decoder else inputs.input_ids.shape[1]

generated_tokens = output_generate.sequences[:, input_length:]

for tok, score in zip(generated_tokens[0], transition_scores[0]):

# | token | token string | log probability | probability

    print(f"| {tok:5d} | {tokenizer.decode(tok):8s} | {score.numpy():.3f} | {np.exp(score.numpy()):.2%}")